<a href="https://colab.research.google.com/github/MasahiroAraki/MachineLearning3/blob/master/notebook/chap12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mlxtend の警告に対処するため、jupyter_client を更新。要ランタイム再起動

In [1]:
!pip install -U scikit-learn jupyter_client --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.3 which is incompatible.
jupyter-kernel-gateway 2.5.2 requires jupyter-client<8.0,>=5.2.0, but you have jupyter-client 8.6.3 which is incompatible.


# 第12章 パターンマイニング

## 頻出項目集合抽出

Aprioriアルゴリズムの実装

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

### 例題12.1

例題12.1 のデータを読み込んで、pandasのDataFrameに変換します。項目がアルファベット順にソートされていることに注意。

In [1]:
# 12章で用いるライブラリ
import numpy as np
import pandas as pd
from scipy.io import arff
import altair as alt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from sklearn.decomposition import NMF

In [2]:
%precision 3
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.precision', 3)

In [3]:
dataset = [['Milk', 'Bread', 'Butter'],
  ['Milk', 'Bread', 'Jam'],
  ['Milk', 'Magazine'],
  ['Bread', 'Butter'],
  ['Milk', 'Bread', 'Butter', 'Jam'],
  ['Magazine'],
  ['Milk', 'Bread', 'Jam', 'Magazine'],
  ['Jam']]

In [4]:
# 疎行列形式の表現を、真偽値を値とする行列に変換
# 項目名がアルファベット順にソートされていることに注意
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
df1

,Bread,Butter,Jam,Magazine,Milk
0,True,True,False,False,True
1,True,False,True,False,True
2,False,False,False,True,True
3,True,True,False,False,False
4,True,True,True,False,True
5,False,False,False,True,False
6,True,False,True,True,True
7,False,False,True,False,False


Aprioriアルゴリズムで3事例以上出現する項目集合を抽出

In [5]:
freq1 = apriori(df1, min_support= 3/len(df1), use_colnames=True)
freq1

,support,itemsets
0,0.625,(Bread)
1,0.375,(Butter)
2,0.500,(Jam)
3,0.375,(Magazine)
4,0.625,(Milk)
5,0.375,"(Butter, Bread)"
6,0.375,"(Jam, Bread)"
7,0.500,"(Milk, Bread)"
8,0.375,"(Milk, Jam)"
9,0.375,"(Milk, Jam, Bread)"


### 例題12.2

supermarket.arff を対象として FPGrowth（Aprioriの高速化版）を試してみます。データをダウンロードし、scipy の loadarff() で Weka のarff 形式のデータを読み込みます。そして pandas の DataFrame で値を文字 t/? から真偽値 True/False に変換し、最終列の total は除きます。

In [6]:
!wget https://git.cms.waikato.ac.nz/weka/weka/-/raw/main/trunk/wekadocs/data/supermarket.arff

--2025-10-10 07:44:00--  https://git.cms.waikato.ac.nz/weka/weka/-/raw/main/trunk/wekadocs/data/supermarket.arff
Resolving git.cms.waikato.ac.nz (git.cms.waikato.ac.nz)... 104.20.29.110, 172.66.153.15, 2606:4700:10::6814:1d6e, ...
Connecting to git.cms.waikato.ac.nz (git.cms.waikato.ac.nz)|104.20.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘supermarket.arff’

supermarket.arff        [         <=>        ]   1.93M   524KB/s    in 3.8s    

2025-10-10 07:44:05 (524 KB/s) - ‘supermarket.arff’ saved [2025871]



In [7]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket.arff')
df = pd.DataFrame(data)
df2 = df.mask(df == b'?', False).mask(df == b't', True)
df2 = df2.drop('total', axis=1)
df2.head()

,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,...,department207,department208,department209,department210,department211,department212,department213,department214,department215,department216
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
df2.iloc[:, :5].head()

,department1,department2,department3,department4,department5
0,False,False,False,False,False
1,True,False,False,False,False
2,False,False,False,False,False
3,True,False,False,False,False
4,False,False,False,False,False


FPGrowthで頻出項目集合を求めます。

In [9]:
freq2 = fpgrowth(df2, min_support=0.3, use_colnames=True, verbose=0)
freq2

/usr/local/lib/python3.12/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.720,(bread and cake)
1,0.640,(fruit)
2,0.640,(vegetables)
3,0.635,(milk-cream)
4,0.604,(baking needs)
...,...,...
100,0.322,"(bread and cake, cheese)"
101,0.322,"(bread and cake, pet foods)"
102,0.317,"(department137, bread and cake)"
103,0.311,"(breakfast food, bread and cake)"


## 規則抽出


### 例題12.3

例題12.2 のデータから Lift値が 1.2 以上の規則を抽出します。

In [10]:
ar2 = association_rules(freq2, metric='lift', min_threshold=1.2)
ar2[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head()

,antecedents,consequents,support,confidence,lift
0,"(fruit, bread and cake)",(vegetables),0.387,0.770,1.204
1,"(bread and cake, vegetables)",(fruit),0.387,0.779,1.217
2,(fruit),"(bread and cake, vegetables)",0.387,0.605,1.217
3,(vegetables),"(fruit, bread and cake)",0.387,0.605,1.204
4,"(milk-cream, fruit)",(vegetables),0.340,0.771,1.205


altairで可視化するためのデータ変換

In [11]:
ar2['antecedents'] = ar2['antecedents'].map(lambda x: ', '.join(x))
ar2['consequents'] = ar2['consequents'].map(lambda x: ', '.join(x))

In [12]:
!pip install vl-convert-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 55.2 MB/s eta 0:00:00


In [13]:
# 基本となる散布図を作成
chart = alt.Chart(ar2).mark_circle().encode(
    x='support',
    y='confidence',
    color='lift',
    tooltip=['antecedents', 'consequents', 'support', 'confidence', 'lift']
).interactive()

# グラフを表示
chart.save('chart.svg')
chart

alt.Chart(...)

## 例題12.4

小規模な映画評価データを使って、行列分解を行います。

<a href="http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/">参考ページ</a>のデータ例を使用します。行がユーザ(5人)、列が映画(4作品)、数値が1-5の5段階評価で、0は評価なしです。

In [14]:
X = np.array([
    [5,3,0,1],
    [4,0,0,1],
    [1,1,0,5],
    [1,0,0,4],
    [0,1,5,4]
])

ここでは、非負値行列分解[NMF](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)を使います。NMFはXを非負の行列WとHに分解します。引数n_componentsは潜在変数の次元数です。

In [15]:
model = NMF(n_components = 2)
W = model.fit_transform(X)
H = model.components_
np.dot(W,H)

array([[5.256, 1.993, 0.   , 1.455],
       [3.504, 1.329, 0.   , 0.97 ],
       [1.313, 0.944, 1.95 , 3.946],
       [0.981, 0.722, 1.528, 3.079],
       [0.   , 0.65 , 2.84 , 5.219]])

空所の値を予測してみます。似たようなユーザを参考に、埋められた値の妥当性を評価してみてください。

人を表す2次元ベクトルを表示します。同じような評価をする人が似たベクトルになっていることを確認してください。

In [16]:
W

array([[0.   , 1.844],
       [0.   , 1.229],
       [0.336, 0.461],
       [0.263, 0.344],
       [0.49 , 0.   ]])

映画を表す2次元ベクトルを表示します。同じような評価をされる映画がないので、それぞれ異なったベクトルになっていることを確認してください。

In [17]:
H.T

array([[ 0.   ,  2.851],
       [ 1.327,  1.081],
       [ 5.798,  0.   ],
       [10.655,  0.789]])

## 演習問題


### 12.1

アプリオリアルゴリズムによる規則抽出のコード例で求めた規則よりも，リフト値の高い規則を探すにはどうすればよいか考察せよ．

FPGrowthでmin_supportを下げて多くの頻出項目集合を求め、association_rulesで規則を評価します。ただしmin_supportをあまり下げすぎると、あまり意味のない項目集合が出てきます。

In [18]:
freq2 = fpgrowth(df2, min_support=0.2, use_colnames=True)
association_rules(freq2, metric="lift", min_threshold=1.5)

/usr/local/lib/python3.12/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(frozen foods, fruit, bread and cake)","(biscuits, vegetables)",0.335,0.381,0.201,0.600,1.574,1.0,0.073,1.547,0.548,0.390,0.354,0.563
1,"(biscuits, bread and cake, vegetables)","(frozen foods, fruit)",0.321,0.402,0.201,0.625,1.553,1.0,0.072,1.593,0.525,0.384,0.372,0.562
2,"(frozen foods, bread and cake, vegetables)","(biscuits, fruit)",0.335,0.397,0.201,0.600,1.512,1.0,0.068,1.508,0.509,0.378,0.337,0.553
3,"(biscuits, fruit)","(frozen foods, bread and cake, vegetables)",0.397,0.335,0.201,0.506,1.512,1.0,0.068,1.346,0.561,0.378,0.257,0.553
4,"(frozen foods, fruit)","(biscuits, bread and cake, vegetables)",0.402,0.321,0.201,0.499,1.553,1.0,0.072,1.355,0.596,0.384,0.262,0.562
5,"(biscuits, vegetables)","(frozen foods, fruit, bread and cake)",0.381,0.335,0.201,0.527,1.574,1.0,0.073,1.406,0.589,0.390,0.289,0.563


### 12.2

scikit-learn を用いて，ベクトル化済みの 20 Newsgroups データを読み込み，NMF を用いて行列分解を行え．その結果として文書を 20 個のトピックに分け，各トピックを代表する 10 語を表示せよ．

In [19]:
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF
import numpy as np

vec = fetch_20newsgroups_vectorized()
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(vec.data) # shape: (n_samples, n_features)

nmf = NMF(n_components=20, random_state=0, init="nndsvda", max_iter=200)
W = nmf.fit_transform(X)
H = nmf.components_

print("W shape:", W.shape)   # (n_samples, 20)
print("H shape:", H.shape)   # (20, n_features)

feature_names = vec.feature_names
n_top_words = 10

for topic_idx, topic in enumerate(H):
    top_features = np.array(feature_names)[topic.argsort()[:-n_top_words - 1:-1]]
    print(f"Topic {topic_idx}: {' '.join(top_features)}")

W shape: (11314, 20)
H shape: (20, 130107)
Topic 0: the of and to in is that on was it
Topic 1: for and to card sale or thanks please with any
Topic 2: god is of that the jesus we to not and
Topic 3: he was his him the to and that had in
Topic 4: edu university cs of posting nntp article host in writes
Topic 5: you to that it is your if are do have
Topic 6: pitt geb gordon banks cs cadre n3jxp dsl chastity shameful
Topic 7: key clipper encryption chip the to escrow keys be government
Topic 8: com netcom article hp writes stratus in re sun inc
Topic 9: scsi drive ide controller bus drives the hard disk isa
Topic 10: keith caltech sgi livesey morality solntze wpd cco jon schneider
Topic 11: nasa gov space henry jpl toronto alaska ___ larc moon
Topic 12: ca team game the hockey games players in nhl play
Topic 13: armenian turkish of armenians the and armenia argic serdar in
Topic 14: israel israeli jews the of arab to jake lebanese in
Topic 15: ohio state magnus acs edu university ryan ma